In [ ]:
# -*- coding: utf-8 -*-
# ======================================================================
# Sección 1: Importación de Bibliotecas
# ======================================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt

# ======================================================================
# Sección 2: Carga y Preparación de Datos
# ======================================================================
# Cargar la base de datos desde el archivo CSV.
try:
    df = pd.read_csv('databases/basedatos_202507161527.csv')
except FileNotFoundError:
    print("Error: El archivo no fue encontrado. Verifique la ruta.")
    exit()

# Definir las variables numéricas clave para el análisis de splines.
features_to_spline = ['Temperatura', 'Pulso', 'PAS', 'PAD', 'SatO2', 'Edad']
# Definir las variables categóricas que también se usarán en el modelo.
categorical_features = ['Sexo', 'Triage']

# Dividir las variables de entrada (X) y la variable objetivo (y).
X = df[features_to_spline + categorical_features]
y = df['Destino']

# Convertir variables categóricas a formato one-hot encoding.
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Dividir los datos en conjuntos de entrenamiento y prueba (80/20).
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# ======================================================================
# Sección 3: ¿Qué son los Splines y por qué usarlos?
# ======================================================================
# Los splines son funciones matemáticas que se utilizan para crear curvas suaves
# que pasan a través de un conjunto de puntos de control (nodos).
# A diferencia de una simple línea recta (regresión lineal), un spline puede
# capturar patrones no lineales en los datos, lo que lo hace ideal para
# modelar relaciones complejas en variables biológicas.
# Por ejemplo, la relación entre el pulso y el destino del paciente podría no
# ser lineal. Los splines permiten que el modelo de regresión capture esta
# curvatura, mejorando la precisión de la predicción.
# En este código, usamos un transformador de splines para crear nuevas variables
# (features) que el modelo de regresión logística puede utilizar.

# ======================================================================
# Sección 4: Implementación de Splines y Modelos
# ======================================================================
# Paso 1: Entrenar un modelo base sin splines para comparación.
print("--- Analizando modelo con datos originales ---")
# Se usa 'lbfgs' porque es robusto para conjuntos de datos grandes.
model_base = LogisticRegression(max_iter=1000, random_state=42)
model_base.fit(X_train, y_train)
y_pred_base = model_base.predict(X_test)
print(classification_report(y_test, y_pred_base))
print("F1-Score (ponderado) del modelo base:", f1_score(y_test, y_pred_base, average='weighted'))

# Paso 2: Crear el transformador de Splines para las variables numéricas.
# Se crea un objeto SplineTransformer para las variables numéricas.
# 'n_knots=5' crea 5 puntos de anclaje que definirán la flexibilidad de la curva.
# 'degree=3' usa polinomios cúbicos para las curvas entre los nodos,
# resultando en una curva suave y continua.
spline_transformer = SplineTransformer(n_knots=5, degree=3, include_bias=False)
X_train_spline = spline_transformer.fit_transform(X_train[features_to_spline])
X_test_spline = spline_transformer.transform(X_test[features_to_spline])

# Paso 3: Combinar las variables transformadas con las categóricas.
# Las variables categóricas no necesitan ser transformadas con splines.
# Se combinan con las nuevas características generadas por el spline.
X_train_spline_combined = np.hstack([X_train_spline, X_train[X_train.columns.difference(features_to_spline)].values])
X_test_spline_combined = np.hstack([X_test_spline, X_test[X_test.columns.difference(features_to_spline)].values])

# Paso 4: Entrenar un modelo con las características de spline.
print("\n--- Analizando modelo con datos transformados por Splines ---")
model_spline = LogisticRegression(max_iter=1000, random_state=42)
model_spline.fit(X_train_spline_combined, y_train)
y_pred_spline = model_spline.predict(X_test_spline_combined)
print(classification_report(y_test, y_pred_spline))
print("F1-Score (ponderado) del modelo con Splines:", f1_score(y_test, y_pred_spline, average='weighted'))

# ======================================================================
# Sección 5: Visualización de un Spline (opcional)
# ======================================================================
# Para ilustrar el efecto del spline, se grafica la transformación de una variable.
# Esto no afecta el modelo, solo es una herramienta visual.
pulso = np.linspace(df['Pulso'].min(), df['Pulso'].max(), 300).reshape(-1, 1)
pulso_spline = spline_transformer.transform(pulso)

plt.figure(figsize=(10, 6))
# Se grafican las nuevas características generadas por el spline.
for i in range(pulso_spline.shape[1]):
    plt.plot(pulso, pulso_spline[:, i], label=f'Spline Feature {i+1}')
plt.title('Transformación de Spline para la Variable Pulso')
plt.xlabel('Pulso (Valor Original)')
plt.ylabel('Valor Transformado (Spline)')
plt.legend()
plt.grid(True)
plt.savefig("spline_pulso.png")
plt.show()
